In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate

In [44]:
data = pd.read_csv('/Users/omarafifi/MyFolders/Differential-Game-Theory-for-SIR-Models/Data/formatted_data.csv')

## Implementing the Adjoint Method

#### 1.1 implement the forward pass 

In [45]:
#step 1 is to solve the forward problem I.e. to integrate the constraining ode. 
timesteps = 21

def SIR(V, t, *args):

    """defines the SIR model
       V = (S, I, R)
       args = (N,Β, γ)
    """

    N, B, g  = args
    S, I, R = V

    dS = -(B/N)*S*I
    dI = (B/N)*S*I - g*I
    dR = g*I

    return dS, dI, dR

def forwardSolve(model, initial_values, params, tf, step):

    """solves the forward problem

       initial_values = (S_0,I_0,R_0)
       params = (N,Β, γ)
       model = SIR
       tf = stop time
       step = number of time steps

    """
    try:
        from scipy.integrate import odeint
        import numpy as np
    except:
        print("unable to install dependancies numpy and scipy.integrate.odeint")
    
    time_steps = np.arange(0, tf, step)
    print(params)

    return odeint(model, t = time_steps, y0 = initial_values, args = params)

### 1.2 Backwards integrate  $ \frac{\partial f }{\partial x}  + \lambda (t)^T( \frac{\partial h }{\partial x} - \frac{\partial }{\partial t} \frac{\partial h }{\partial \dot x}) - \frac{\partial \lambda }{\partial t} \frac{\partial h }{\partial \dot x} = 0 $ from $t_f \to 0$

In [46]:
from scipy.interpolate import interp1d


#dummy values for I and I hat 
I = data['I'].to_numpy()
S = data['S'].to_numpy()
I_hat = np.arange(0, 22, 1)

I = interp1d(np.arange(0, 22, 1), I)
S_hat = interp1d(np.arange(0, 22, 1), S)
I_hat = interp1d(np.arange(0, 22, 1), I_hat)

def lamdaDE(λ, t,  *args):
    """
    calculate the derivative of the I component of lambda 
    infections should be a tuple (I, I_pred) of ground truth and predicted values
    over the given time frame. 

    args = (N,Β, γ)
    """

    N, β, γ  = args

    df_dx = 2*np.array([0, I(t) - I_hat(t), 0])

    dh_dx = np.array([[β*I_hat(t)/N ,  β*S_hat(t)/N, 0],
                     [-β*I_hat(t)/N, -β*S_hat(t)/N, 0],
                     [0            ,            -γ, 0]])

    return  df_dx + np.array(λ).T@dh_dx

def backwardSolve(model, params, tf):
    """Backwards solve for λ(t). 
    """
    #we have to integrate backwards

    from scipy.integrate import odeint
    import numpy as np

    time_steps = np.arange(tf -1, -1, -1)

    return odeint(func = lamdaDE, y0 = [0,0,0], t = time_steps, args = params)


3) #### Integrate for $d_pF$


$$d_pF(x,p) = \int_0^{t_f} \left ( \frac{\partial f}{\partial p} + \lambda(t)^T  
\frac{\partial h}{\partial p} \right )dt + \lambda(0)^T  \left [ \frac{\partial g}{\partial x_0} \right ]^{-1}
\left [ \frac{\partial g}{\partial p} \right ]$$

In [ ]:
df_dp = np.zeros((5))

dg_dx0 = np.eye(3)
dg_dp = -np.array([[0,0,1,0,0],
                   [0,0,0,1,0],
                   [0,0,0,0,1]])
dg_dp

N = 1000

def dh_dp(t):
    return np.array([[ I_hat(t)*S_hat(t) ,         0,0,0,0],
                    [-I_hat(t)*S_hat(t)/N,  I_hat(t),0,0,0],
                    [ 0                  , -I_hat(t),0,0,0]])

def integrand(t,λ): 
    return df_dp + λ[t].T@dh_dp(t)


def dF_dp(tf, λ):
 
    #we get all the timesteps: 
    time_steps = np.arange(0, tf, 1)
    integrands = [integrand(t) for t in time_steps]

    #integrate with simpson's method
    return integrate.simpson(y = integrands, x = time_stpes, axis = 0)






